<a href="https://colab.research.google.com/github/yachu33/Malicious-PEfiles-clustering/blob/master/Cluster_malicious_pefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HW04 Static Analysis

Since we cannot share real malware binary with you. We give you the pefile analysis result (by calling pefile_dump() ) and save the result in a dictionary. You have to practice writing a parser before analyzing the given pefile dump file.

### Download data


In [2]:
# Here is the dump dictionary.
!wget https://drive.google.com/uc?id=1BTr1IDPhRF9WnFjXFwJWiqJ31qa-EWL9 -O pef_dumpdict_40.pickle

--2020-04-14 15:23:33--  https://drive.google.com/uc?id=1BTr1IDPhRF9WnFjXFwJWiqJ31qa-EWL9
Resolving drive.google.com (drive.google.com)... 172.217.212.139, 172.217.212.138, 172.217.212.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.212.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8ndigvb0sv6r0ifr9ml7pq6nshp70sjd/1586877750000/17174434063503717705/*/1BTr1IDPhRF9WnFjXFwJWiqJ31qa-EWL9 [following]
--2020-04-14 15:23:33--  https://doc-08-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8ndigvb0sv6r0ifr9ml7pq6nshp70sjd/1586877750000/17174434063503717705/*/1BTr1IDPhRF9WnFjXFwJWiqJ31qa-EWL9
Resolving doc-08-38-docs.googleusercontent.com (doc-08-38-docs.googleusercontent.com)... 108.177.111.132, 2607:f8b0:4001:c07::84
Connecting to doc-08-38-docs.googleusercontent.com (doc-08-38-docs.googleusercontent.com)|1

In [3]:
# The size should be 414677 bytes.
!ls -al

total 424
drwxr-xr-x 1 root root   4096 Apr 14 15:23 .
drwxr-xr-x 1 root root   4096 Apr 14 15:20 ..
drwxr-xr-x 1 root root   4096 Apr  9 16:10 .config
-rw-r--r-- 1 root root 414677 Apr 14 15:23 pef_dumpdict_40.pickle
drwxr-xr-x 1 root root   4096 Apr  3 16:24 sample_data


In [4]:
# It is stored in a pickle file.
# You can retrive it by using the following code.

import pickle
import os 

with open("/content/pef_dumpdict_40.pickle", 'rb') as fd:
  pef_dumpdicts = pickle.load(fd)
  print(pef_dumpdicts.keys())

dict_keys(['07edeebc14b053fffdfb4dcc08959360', '0cfab9961285ab6e04f449c6d09cec37', '2c1e90810ca30865cbf4440118c4aa0f', '412c8d4d6afc88391e6dad508106681a', '656f628856a62f1dc14748bff4cee2e5', '8595c554539a172534c95222260650e5', '88b666fd4ef1c21f4051ad3f859ee5bd', '8aa66c2a430c3545c542e8419de13385', 'e776d8d8d187bdb68f41dc9ccfc86444', 'ed1304342c5794cb0a55840253793b45', '2650e58a609438ba18706f539b49a6af', '2a5932a800123709eff21dbd2310e090', '4271ba51876b1e931dd55b2ccff25789', '52cae62ada8b02adae128e35686cbc16', '774b71a232568ca7ea09c0a26b82f8e5', '7b57508defc0934c69b98544b4bac185', '8081b6b58cc3372ad62a5d8b6ce09c0b', '8e6d8d46e4a7482b9475e6c5b211dc61', 'b4e89348aa9b635b8edce746f8231303', 'e0ad769d1bd2b9b6444796f99b52a43f', '0c6e68a34804df907df26d7139b1f052', '0e751a5b6db31895c283ddd9757477de', '0f74a58af435599ec94eff2b72d5e55e', '1d0ce31c6d7129a74e2921e8d161c752', '2cceeae738cae520544e9d9fda9ca30e', '2ea5285f04bc9a0b3008165779a382ec', '2f45c7132f623cd7caea6d7965d53a0f', '3b569cd1c63b8344

### A dump file example


In [5]:
# Get a 32-bit putty pe file
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe -O putty32.exe

--2020-04-14 15:23:44--  https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe
Resolving the.earth.li (the.earth.li)... 93.93.131.124, 2a00:1098:86:4d:c0ff:ee:15:900d
Connecting to the.earth.li (the.earth.li)|93.93.131.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://the.earth.li/~sgtatham/putty/0.73/w32/putty.exe [following]
--2020-04-14 15:23:44--  https://the.earth.li/~sgtatham/putty/0.73/w32/putty.exe
Reusing existing connection to the.earth.li:443.
HTTP request sent, awaiting response... 200 OK
Length: 1096080 (1.0M) [application/x-msdos-program]
Saving to: ‘putty32.exe’

putty32.exe         100%[===================>]   1.04M  1.34MB/s    in 0.8s    

2020-04-14 15:23:45 (1.34 MB/s) - ‘putty32.exe’ saved [1096080/1096080]



In [6]:
!pip install pefile

     |████████████████████████████████| 71kB 2.4MB/s 
  Created wheel for pefile: filename=pefile-2019.4.18-cp36-none-any.whl size=60824 sha256=dfd5ed10d7ceddbbdb8df9797987d922f38e6116beb2fc0f9d784b6ce3283bc1
  Stored in directory: /root/.cache/pip/wheels/1c/a1/95/4f33011a0c013c872fe6f0f364dc463a2588120820e40a30d8
Successfully built pefile


In [0]:
import pefile

pef = pefile.PE("/content/putty32.exe", True)
dump_dict = pef.dump_dict()

In [8]:
# a pefile dump dictionary has some information
# they are stored in different "keys"
print(dump_dict.keys())

dict_keys(['DOS_HEADER', 'NT_HEADERS', 'FILE_HEADER', 'Flags', 'OPTIONAL_HEADER', 'DllCharacteristics', 'PE Sections', 'Directories', 'Version Information', 'Imported symbols', 'Resource directory', 'LOAD_CONFIG', 'Base relocations'])


In [9]:
# show one of the data in 'DOS_HEADER' key
# and it looks like a python dictionary
# you can put this data in a json or a dictionaary structure for further analysis
print(dump_dict['DOS_HEADER'])

{'Structure': 'IMAGE_DOS_HEADER', 'e_magic': {'FileOffset': 0, 'Offset': 0, 'Value': 23117}, 'e_cblp': {'FileOffset': 2, 'Offset': 2, 'Value': 120}, 'e_cp': {'FileOffset': 4, 'Offset': 4, 'Value': 1}, 'e_crlc': {'FileOffset': 6, 'Offset': 6, 'Value': 0}, 'e_cparhdr': {'FileOffset': 8, 'Offset': 8, 'Value': 4}, 'e_minalloc': {'FileOffset': 10, 'Offset': 10, 'Value': 0}, 'e_maxalloc': {'FileOffset': 12, 'Offset': 12, 'Value': 0}, 'e_ss': {'FileOffset': 14, 'Offset': 14, 'Value': 0}, 'e_sp': {'FileOffset': 16, 'Offset': 16, 'Value': 0}, 'e_csum': {'FileOffset': 18, 'Offset': 18, 'Value': 0}, 'e_ip': {'FileOffset': 20, 'Offset': 20, 'Value': 0}, 'e_cs': {'FileOffset': 22, 'Offset': 22, 'Value': 0}, 'e_lfarlc': {'FileOffset': 24, 'Offset': 24, 'Value': 64}, 'e_ovno': {'FileOffset': 26, 'Offset': 26, 'Value': 0}, 'e_res': {'FileOffset': 28, 'Offset': 28, 'Value': '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'}, 'e_oemid': {'FileOffset': 36, 'Offset': 36, 'Value': 0}, 'e_oeminfo': {'FileOffset': 

In [10]:
# for example, I put it in a dictionaty
# now I can easily retrive the info stored in it by keys
dos_header_dict = dict(dump_dict['DOS_HEADER'])
print(dos_header_dict.keys())

print(dos_header_dict['Structure'])  # name
print(dos_header_dict['e_magic'])  # e_magic

dict_keys(['Structure', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr', 'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs', 'e_lfarlc', 'e_ovno', 'e_res', 'e_oemid', 'e_oeminfo', 'e_res2', 'e_lfanew'])
IMAGE_DOS_HEADER
{'FileOffset': 0, 'Offset': 0, 'Value': 23117}


In [11]:
# another example

file_header_dict = dict(dump_dict['FILE_HEADER'])
print(file_header_dict.keys())

print(file_header_dict['Machine'])
print(file_header_dict['NumberOfSections'])
print(file_header_dict['TimeDateStamp'])

dict_keys(['Structure', 'Machine', 'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable', 'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics'])
{'FileOffset': 124, 'Offset': 0, 'Value': 332}
{'FileOffset': 126, 'Offset': 2, 'Value': 7}
{'FileOffset': 128, 'Offset': 4, 'Value': '0x5D873EBE [Sun Sep 22 09:28:30 2019 UTC]'}


In [0]:
# You can even print all dump_dict to find some valueable infomation.
# pprint stands for pretty print, which makes you easier to view the data
import pprint
# pprint.pprint(dump_dict['Imported symbols']) # change this to other keys
# pprint.pprint(dump_dict['DOS_HEADER']) # change this to other keys
# pprint.pprint(pef_dumpdicts['0cfab9961285ab6e04f449c6d09cec37'])

### Analyze the downloaded data

In [13]:
# Now pef_dumpdict is what we want you to analyze.
# It combines 40 malware's pefile.dump_dict().

print(len(pef_dumpdicts))

40


In [14]:
# The id of the 40 malwares are
for key in pef_dumpdicts.keys():
  print(key)

07edeebc14b053fffdfb4dcc08959360
0cfab9961285ab6e04f449c6d09cec37
2c1e90810ca30865cbf4440118c4aa0f
412c8d4d6afc88391e6dad508106681a
656f628856a62f1dc14748bff4cee2e5
8595c554539a172534c95222260650e5
88b666fd4ef1c21f4051ad3f859ee5bd
8aa66c2a430c3545c542e8419de13385
e776d8d8d187bdb68f41dc9ccfc86444
ed1304342c5794cb0a55840253793b45
2650e58a609438ba18706f539b49a6af
2a5932a800123709eff21dbd2310e090
4271ba51876b1e931dd55b2ccff25789
52cae62ada8b02adae128e35686cbc16
774b71a232568ca7ea09c0a26b82f8e5
7b57508defc0934c69b98544b4bac185
8081b6b58cc3372ad62a5d8b6ce09c0b
8e6d8d46e4a7482b9475e6c5b211dc61
b4e89348aa9b635b8edce746f8231303
e0ad769d1bd2b9b6444796f99b52a43f
0c6e68a34804df907df26d7139b1f052
0e751a5b6db31895c283ddd9757477de
0f74a58af435599ec94eff2b72d5e55e
1d0ce31c6d7129a74e2921e8d161c752
2cceeae738cae520544e9d9fda9ca30e
2ea5285f04bc9a0b3008165779a382ec
2f45c7132f623cd7caea6d7965d53a0f
3b569cd1c63b83446420dd798a8622b7
3f07e530430d1d450bfeaf0a595109b1
4aa9b2104ac7c7602c4258a0f7ce2b44
17af47cdfd

In [15]:
# so the fist dump_dict is
first_dict = pef_dumpdicts['07edeebc14b053fffdfb4dcc08959360']

# you can now access the data you want
# for example print all sections and their info.
for sec in first_dict['PE Sections']:
  print(sec)

# or like this (as what we see in pefile demonstration)
print("\nName, Misc_VirtualSize, SizeOfRawData, Entropy, MD5")
for sec in first_dict['PE Sections']:
  print(sec['Name']['Value'], sec['Misc_VirtualSize']['Value'], sec['SizeOfRawData']['Value'], sec['Entropy'], sec['MD5'])

{'Structure': 'IMAGE_SECTION_HEADER', 'Name': {'FileOffset': 464, 'Offset': 0, 'Value': 'UPX0\\x00\\x00\\x00\\x00'}, 'Misc': {'FileOffset': 472, 'Offset': 8, 'Value': 20480}, 'Misc_PhysicalAddress': {'FileOffset': 472, 'Offset': 8, 'Value': 20480}, 'Misc_VirtualSize': {'FileOffset': 472, 'Offset': 8, 'Value': 20480}, 'VirtualAddress': {'FileOffset': 476, 'Offset': 12, 'Value': 4096}, 'SizeOfRawData': {'FileOffset': 480, 'Offset': 16, 'Value': 0}, 'PointerToRawData': {'FileOffset': 484, 'Offset': 20, 'Value': 1024}, 'PointerToRelocations': {'FileOffset': 488, 'Offset': 24, 'Value': 0}, 'PointerToLinenumbers': {'FileOffset': 492, 'Offset': 28, 'Value': 0}, 'NumberOfRelocations': {'FileOffset': 496, 'Offset': 32, 'Value': 0}, 'NumberOfLinenumbers': {'FileOffset': 498, 'Offset': 34, 'Value': 0}, 'Characteristics': {'FileOffset': 500, 'Offset': 36, 'Value': 3758096512}, 'Flags': ['IMAGE_SCN_CNT_UNINITIALIZED_DATA', 'IMAGE_SCN_MEM_EXECUTE', 'IMAGE_SCN_MEM_READ', 'IMAGE_SCN_MEM_WRITE'], 'Entr

In [16]:
# Here you can output the dll and windows api used by a pe file
# You may print dump_dict['Imported symbols'] first
# and try to locate the info you want to retrive.

api_dict = dict()
for records in first_dict['Imported symbols']:
  for record in records:
    if 'Name' in record:
      if isinstance(record['Name'], bytes):
        dll = record['DLL'].decode("ascii")
        name = record['Name'].decode("ascii")
        if dll not in api_dict:
          api_dict[dll] = set()
        api_dict[dll].add(name)

for dll in api_dict:
  print(dll)
  print("", api_dict[dll])

KERNEL32.DLL
 {'LoadLibraryA', 'GetProcAddress', 'ExitProcess'}
ADVAPI32.dll
 {'RegCloseKey'}
MSVCRT.dll
 {'rand'}
USER32.dll
 {'wsprintfA'}
WININET.dll
 {'InternetOpenA'}


### Extract more information

Now its your turn.
Try to buld a feature table for these 40 malwares.
Here is an example for api call.

In [0]:
api_set = set()
for key in pef_dumpdicts.keys(): # iter 40 malwares
  pef_dump_dict = pef_dumpdicts[key]
  for records in pef_dump_dict['Imported symbols']:
    for record in records:
      if 'Name' in record:
        if isinstance(record['Name'], bytes):
          name = record['Name'].decode("ascii")
          api_set.add(name)

api_list = list(api_set) 

In [18]:
api_list

['wsprintfA',
 'InternetOpenA',
 'ExitProcess',
 'GetProcAddress',
 'rand',
 'LoadLibraryA',
 'RegCloseKey']

Now you find an interesting thing, that all 40 malwares calls these 7 APIs. So it is impossible to distinguish them by using these 7 APIs. So please try to use some other features.

Hint #1: these malwares are packed. UPX may unpacked them.

Hint #2: these 40 malware samples are from 4 families. Can you classify or cluster them into 4 groups?

### Your solution

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame()
df['keys'] = pef_dumpdicts.keys()

~Machine~

In [0]:
machine_list = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  machine_list.append(pef_dump_dict['FILE_HEADER']['Machine']['Value'])
# all same

1.Entropy

In [0]:
entropy_list = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  entropy = []
  for sec in pef_dump_dict['PE Sections']:
    entropy.append(sec['Entropy'])

  entropy_list.append(max(entropy))
df['entropy'] = entropy_list

2.Num of section

In [0]:
num_sec = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  num_sec.append(pef_dump_dict['FILE_HEADER']['NumberOfSections']['Value'])
df['num_sec'] = num_sec

3.Time stamp

In [0]:
import re
import datetime

time_stamp = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  timeDate = pef_dump_dict['FILE_HEADER']['TimeDateStamp']['Value']
  time_fmt = re.search('\[(.+?)\]', timeDate).group(1)
  timest = datetime.datetime.strptime(time_fmt, '%a %b %d %H:%M:%S %Y %Z').timestamp()
  time_stamp.append(timest)

df['time_stamp'] = time_stamp

4.AddressOfEntryPoint

In [0]:
addr_entryP = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  addr_entryP.append(pef_dump_dict['OPTIONAL_HEADER']['AddressOfEntryPoint']['Value'])
df['addr_entryP'] = addr_entryP

5.size of row data 

In [0]:
size_rowD = []
for key in pef_dumpdicts.keys():
  pef_dump_dict = pef_dumpdicts[key]
  rowD = []
  for sec in pef_dump_dict['PE Sections']:
    rowD.append(sec['SizeOfRawData']['Value'])

  size_rowD.append(max(rowD))
df['size_rowD'] = size_rowD

MD5

In [0]:
md5_set = set()
for key in pef_dumpdicts.keys(): # iter 40 malwares
  pef_dump_dict = pef_dumpdicts[key]
  for sec in pef_dump_dict['PE Sections']:
    md5_set.add(sec['MD5'])

md5_set = list(md5_set)

In [0]:
def checkIdentical(chList):
  return len(set(chList)) <= 1

In [0]:
md5_pd = pd.DataFrame()
# print(md5_set[1])
for i in range(len(md5_set)):
  md5_temp = []
  for key in pef_dumpdicts.keys():
    pef_dump_dict = pef_dumpdicts[key]
    ex = 0
    for sec in pef_dump_dict['PE Sections']:
      if sec['MD5'] == md5_set[i]:
        ex = 1
        break
    md5_temp.append(ex)
  if not (checkIdentical(md5_temp)): #remove feature if values are all same
    df['md5_'+ str(i)] = md5_temp

Clustering

In [30]:
df

,keys,entropy,num_sec,time_stamp,addr_entryP,size_rowD,md5_0,md5_1,md5_2,md5_3,md5_4,md5_5,md5_6,md5_7,md5_8,md5_9,md5_10,md5_11,md5_12,md5_13,md5_14,md5_15,md5_16,md5_17,md5_18,md5_19,md5_20,md5_21,md5_22,md5_23,md5_24,md5_25,md5_26,md5_27,md5_28,md5_29,md5_30,md5_31,md5_33,md5_34,md5_35,md5_36,md5_37
0,07edeebc14b053fffdfb4dcc08959360,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,0cfab9961285ab6e04f449c6d09cec37,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,2c1e90810ca30865cbf4440118c4aa0f,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
3,412c8d4d6afc88391e6dad508106681a,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,656f628856a62f1dc14748bff4cee2e5,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
5,8595c554539a172534c95222260650e5,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
6,88b666fd4ef1c21f4051ad3f859ee5bd,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
7,8aa66c2a430c3545c542e8419de13385,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
8,e776d8d8d187bdb68f41dc9ccfc86444,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
9,ed1304342c5794cb0a55840253793b45,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0


In [31]:
from sklearn import cluster

pef_X = df.drop(['keys'], axis=1)
pef_X
kmeans_fit = cluster.KMeans(n_clusters = 4).fit(pef_X)
kmeans_fit.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3], dtype=int32)

In [32]:
df.insert(loc=0, column='Families', value=kmeans_fit.labels_)
df

,Families,keys,entropy,num_sec,time_stamp,addr_entryP,size_rowD,md5_0,md5_1,md5_2,md5_3,md5_4,md5_5,md5_6,md5_7,md5_8,md5_9,md5_10,md5_11,md5_12,md5_13,md5_14,md5_15,md5_16,md5_17,md5_18,md5_19,md5_20,md5_21,md5_22,md5_23,md5_24,md5_25,md5_26,md5_27,md5_28,md5_29,md5_30,md5_31,md5_33,md5_34,md5_35,md5_36,md5_37
0,2,07edeebc14b053fffdfb4dcc08959360,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,2,0cfab9961285ab6e04f449c6d09cec37,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,2,2c1e90810ca30865cbf4440118c4aa0f,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
3,2,412c8d4d6afc88391e6dad508106681a,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
4,2,656f628856a62f1dc14748bff4cee2e5,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
5,2,8595c554539a172534c95222260650e5,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
6,2,88b666fd4ef1c21f4051ad3f859ee5bd,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
7,2,8aa66c2a430c3545c542e8419de13385,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
8,2,e776d8d8d187bdb68f41dc9ccfc86444,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
9,2,ed1304342c5794cb0a55840253793b45,7.902853,3,1.088239e+09,33280,7680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
